In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD
from keras.utils import plot_model
from keras import regularizers
from keras.callbacks import EarlyStopping


In [2]:
dataset = pd.read_csv("obesity.csv")
print("Classes in the dataset:", dataset['NObeyesdad'].unique())

Classes in the dataset: ['Normal_Weight' 'Overweight_Level_I' 'Overweight_Level_II'
 'Obesity_Type_I' 'Insufficient_Weight' 'Obesity_Type_II'
 'Obesity_Type_III']


##One hot encoding + Scaling Data

In [3]:
categories = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'CAEC', 'CALC', 'MTRANS', 'NObeyesdad']
for c in categories:
  dummies = pd.get_dummies(dataset[c], prefix=(str(c)+"_"))
  dataset = pd.concat([dataset, dummies], axis=1)
dataset = dataset.drop(columns=categories)

In [4]:
d = ['NObeyesdad__Insufficient_Weight',
       'NObeyesdad__Normal_Weight', 'NObeyesdad__Obesity_Type_I',
       'NObeyesdad__Obesity_Type_II', 'NObeyesdad__Obesity_Type_III',
       'NObeyesdad__Overweight_Level_I', 'NObeyesdad__Overweight_Level_II']
X = dataset.drop(columns=d)
y = dataset[d]

In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_rescaled = scaler.fit_transform(X)
X = pd.DataFrame(data = X_rescaled, columns = X.columns)

In [28]:
data_train, data_test, class_train, class_test = train_test_split(X, y, test_size=0.2, random_state=15)

Model with using only Dropout Techniques

In [64]:
def createModel():
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim = 31))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(7, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
model = createModel()
model.fit(data_train, class_train, epochs=100, batch_size=20)

Epoch 1/100
85/85 [==============================] - 1s 2ms/step - loss: 1.7768 - accuracy: 0.3945
Epoch 2/100
85/85 [==============================] - 0s 2ms/step - loss: 1.3101 - accuracy: 0.5474
Epoch 3/100
85/85 [==============================] - 0s 2ms/step - loss: 1.0710 - accuracy: 0.6001
Epoch 4/100
85/85 [==============================] - 0s 2ms/step - loss: 0.9437 - accuracy: 0.6546
Epoch 5/100
85/85 [==============================] - 0s 2ms/step - loss: 0.8423 - accuracy: 0.6919
Epoch 6/100
85/85 [==============================] - 0s 2ms/step - loss: 0.7787 - accuracy: 0.7139
Epoch 7/100
85/85 [==============================] - 0s 2ms/step - loss: 0.7044 - accuracy: 0.7518
Epoch 8/100
85/85 [==============================] - 0s 2ms/step - loss: 0.6613 - accuracy: 0.7666
Epoch 9/100
85/85 [==============================] - 0s 2ms/step - loss: 0.5939 - accuracy: 0.7944
Epoch 10/100
85/85 [==============================] - 0s 2ms/step - loss: 0.5428 - accuracy: 0.8081
Epoch 11/

In [65]:
test_loss, test_acc = model.evaluate(data_test, class_test, verbose=0)
print('Testing Loss:', test_loss)
print('Testing Accuracy:', test_acc)

Testing Loss: 0.2156379073858261
Testing Accuracy: 0.9550827145576477


In [66]:
train_loss, train_acc = model.evaluate(data_train, class_train, verbose=0)
print('Training Loss:', train_loss)
print('Training Accuracy:', train_acc)

Training Loss: 0.011413161642849445
Training Accuracy: 0.9976303577423096


Looking at the results, the training loss is lower than the testing loss by a good amount, so this is a possible sign of overfitting. So to regularize any overfitting, we tried adding dropout and batch normalization.

In [67]:
#Neural Network
def createModel():
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim = 31))
  BatchNormalization()
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  BatchNormalization()
  model.add(Dropout(0.5))
  model.add(Dense(7, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
model2 = createModel()
h = model2.fit(data_train, class_train, epochs=100, batch_size=20)

Epoch 1/100
85/85 [==============================] - 1s 2ms/step - loss: 1.8741 - accuracy: 0.2541
Epoch 2/100
85/85 [==============================] - 0s 2ms/step - loss: 1.6920 - accuracy: 0.3602
Epoch 3/100
85/85 [==============================] - 0s 2ms/step - loss: 1.5605 - accuracy: 0.3993
Epoch 4/100
85/85 [==============================] - 0s 2ms/step - loss: 1.4447 - accuracy: 0.4336
Epoch 5/100
85/85 [==============================] - 0s 2ms/step - loss: 1.3402 - accuracy: 0.4893
Epoch 6/100
85/85 [==============================] - 0s 2ms/step - loss: 1.2771 - accuracy: 0.5018
Epoch 7/100
85/85 [==============================] - 0s 2ms/step - loss: 1.2444 - accuracy: 0.5278
Epoch 8/100
85/85 [==============================] - 0s 2ms/step - loss: 1.2083 - accuracy: 0.5024
Epoch 9/100
85/85 [==============================] - 0s 2ms/step - loss: 1.1325 - accuracy: 0.5616
Epoch 10/100
85/85 [==============================] - 0s 2ms/step - loss: 1.1120 - accuracy: 0.5527
Epoch 11/

In [68]:
test_loss, test_acc = model2.evaluate(data_test, class_test, verbose=0)
print('Loss:', test_loss)
print('Accuracy:', test_acc)

Loss: 0.2633262276649475
Accuracy: 0.9267139434814453


In [69]:
train_loss, train_acc = model2.evaluate(data_train, class_train, verbose=0)
print('Training Loss:', train_loss)
print('Training Accuracy:', train_acc)

Training Loss: 0.12937034666538239
Training Accuracy: 0.9697867035865784


After applying the Dropout technique for 50% of the neurons and batch normalization to stabilize the training process, the test loss is closer to the training loss now, while performing well at 92.7%. Therefore while this model may have a higher loss than the first one, it is better in comparison to its corrresponding training loss and accuracy.



In [71]:
dataset_copy = pd.read_csv("obesity.csv")

Cross - Validation for the DNN

In [73]:
#Cross Validation for DNN - https://stackoverflow.com/questions/48085182/cross-validation-in-keras
from sklearn.model_selection import StratifiedKFold
def train_evaluate(model, xtrain, ytrain, xtest, ytest):
  model.fit(data_train, class_train, epochs=100, batch_size=20)
  test_loss, test_acc = model.evaluate(data_test, class_test, verbose=0)
  return test_loss, test_acc
kFold = StratifiedKFold(n_splits = 10)
i=0
sum=0
temp = dataset_copy["NObeyesdad"]
for train, test in kFold.split(X,temp):
  print()
  model = None
  model = createModel()
  loss, acc = train_evaluate(model,X.iloc[[train[0]]],y.iloc[[train[0]]],X.iloc[[test[0]]],y.iloc[[test[0]]])
  print("Fold i Accuracy: ", acc)
  i+=1
  sum += acc

print("Mean accuracy:", (sum/i))
#0.9141843974590301


Epoch 1/100
85/85 [==============================] - 1s 2ms/step - loss: 1.9121 - accuracy: 0.2133
Epoch 2/100
85/85 [==============================] - 0s 2ms/step - loss: 1.7272 - accuracy: 0.3359
Epoch 3/100
85/85 [==============================] - 0s 2ms/step - loss: 1.5897 - accuracy: 0.3993
Epoch 4/100
85/85 [==============================] - 0s 2ms/step - loss: 1.4599 - accuracy: 0.4378
Epoch 5/100
85/85 [==============================] - 0s 2ms/step - loss: 1.3798 - accuracy: 0.4668
Epoch 6/100
85/85 [==============================] - 0s 2ms/step - loss: 1.2972 - accuracy: 0.4828
Epoch 7/100
85/85 [==============================] - 0s 2ms/step - loss: 1.2575 - accuracy: 0.5219
Epoch 8/100
85/85 [==============================] - 0s 2ms/step - loss: 1.1900 - accuracy: 0.5432
Epoch 9/100
85/85 [==============================] - 0s 2ms/step - loss: 1.1341 - accuracy: 0.5610
Epoch 10/100
85/85 [==============================] - 0s 2ms/step - loss: 1.1156 - accuracy: 0.5646
Epoch 11

After running Cross-Validation on the model with dropout and Batch Normalization, the mean accuracy was 91.4% which is good, and one that may be a possibility as our best model, but the loss value is a factor to consider.